# Compare TIFF images generated for HistoCat and CPOUT

This script checks to see if the channel images from HistoCat and CPOUT for each acquisition are the same. Images haven't gone through hot pixel filtration.

In [1]:
import numpy as np
import pandas as pd
import tifffile
from os import PathLike
from pathlib import Path

### Specify directory storing histocat images and stacked images

In [2]:
histo_dir = "/Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/histocat"
histo_dir = Path(histo_dir)
cpout_dir = "/Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images"
cpout_dir = Path(cpout_dir)

### Identify acquisitions for each MCD (slide)

In [3]:
histo_acqs = list(histo_dir.glob("[!.]*")) #a list object with Posix Path to each acquisition folder in histocat
equal = {}
not_equal = {}
dict_csv = {}
dict_img = {}
for acq in histo_acqs:
    equal_channels = []
    not_equal_channels = []
    acq_id = acq.stem
    acq_histo_path = histo_dir / acq_id
    for i in cpout_dir.glob("[!.]*"):
        if acq_id in str(i):
            if i.suffix == ".csv":
                channel_file = pd.read_csv(i,header=None, index_col=None)
                dict_csv[acq_id] = i #sanity check
            if i.suffix == ".tiff":
                stacked_img = tifffile.imread(i)
                dict_img[acq_id] = i #sanity check
                assert stacked_img.ndim == 3
                assert stacked_img.shape[0] == len(channel_file)
    for index, channel in channel_file.iterrows():
        channel_name = channel[0]
        cpout_channel_img = stacked_img[index]
        for histo_acq_channel in acq_histo_path.iterdir():
            if channel_name in str(histo_acq_channel):
                histo_channel_img = tifffile.imread(histo_acq_channel)    
                #assert np.array_equal(histo_channel_img, cpout_channel_img) == True
                if np.array_equal(histo_channel_img, cpout_channel_img) == True:
                    equal_channels.append(channel_name)
                if np.array_equal(histo_channel_img, cpout_channel_img) == False:
                    not_equal_channels.append(channel_name)
    equal[acq_id]=equal_channels
    not_equal[acq_id]=not_equal_channels
                
        


In [5]:
for acq in histo_acqs:
    acq_id = acq.stem
    print(acq_id)
    for i in cpout_dir.glob("[!.]*"):
        if acq_id in str(i):
            if i.suffix == ".csv":
                u=i
                print("csv file is ",i)
            if i.suffix == ".tiff":
                print("tiff file is ",i)
    print("current file is ",u)
        
                
            
        
        

Patient4_s0_a5_ac
csv file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/Patient4_s0_a5_ac_full_noHPF.csv
tiff file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/Patient4_s0_a5_ac_full_noHPF.tiff
current file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/Patient4_s0_a5_ac_full_noHPF.csv
Patient1_s0_a3_ac
tiff file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/Patient1_s0_a3_ac_full_noHPF.tiff
csv file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/Patient1_s0_a3_ac_full_noHPF.csv
current file is  /Users/bonocheong/Desktop/PhD_Pre-readings/Imaging_Based Mass_Cytometry/ImcSegmentationPipeline/analysis/cpout/images/P